Function that returns 2-dimensional boolean array, random or ordered. Each boolean digit corresponds to up- and down-spin.

**initialize** (Length, initial_state="random")

In [ ]:
function initialize(Length::Int64, initial_state="random")::Array{Bool}

    if initial_state == "random"
        Config = rand(Bool, Length,Length)
    elseif initial_state == "ordered"
        Config = ones(Bool, Length,Length)
    end

    return Config
end

Functions that make spin configuration undergo certain time of stochastic dynamics.

**evolution_rs** (Config, Temperature, Time): 	*Metropolis, random-site flipping*

**evolution_tw** (Config, Temperature, Time):	*Metropolis, typewriter flipping*

Function inputs

>**Config**:	2-dimensional boolean array of spin
>
>**Temperature**:	temperature of the stochastic dynamics
>
>**Time**:	duration of the dynamics. $\text{Time} \times \text{Length}^2$ spin-flipping attempts will be made

---------------------------------------

Functions that return a vector of sampled magnetization and energy data. Each function has distinct Markov chain.

**stream_rs** (Config, Temperature, num_sample, num_sample_burn, period_sample): *Metropolis, random-site flipping*

**stream_tw** (Config, Temperature, num_sample, num_sample_burn, period_sample): *Metropolis, typewriter flipping*

**stream_glauber** (Config, Temperature, num_sample, num_sample_burn, period_sample):   *Glauber, random-site flipping*

**stream_glauber_tw** (Config, Temperature, num_sample, num_sample_burn, period_sample):	*Glauber, typewriter flipping*

Function inputs

>**Config**:	2-dimensional boolean array of spin
>
>**Temperature**:	temperature of the stochastic dynamics
>
>**num_sample**:	number of sample physical properties
>
>**num_sample_burn**:	After this time the stream of sample is taken
>
>**period_sample**:	time between each sample

In [ ]:
# returns neighboring sites with periodic boundary condition
# 2-dimensional NxN system
function x1x2y1y2(x::Int64,y::Int64,N::Int64)
    if x == 1
        x1 = N
    else
        x1 = x-1
    end
    if x == N
        x2 = 1
    else
        x2 = x+1
    end         
    if y == 1
        y1 = N
    else
        y1 = y-1
    end
    if y == N
        y2 = 1
    else
        y2 = y+1
    end
    return x1, x2, y1, y2
end

In [ ]:
function dynamics_rs(Config::Array{Bool}, Temperature, Time::Int64)
    Length = size(Config,1)
    list_exp = exp.(2*[-4 -2] / Temperature)

    for time = 1:Time
        for iteration = 1:Length^2
            x = mod(rand(Int8),Length) + 1
            y = mod(rand(Int8),Length) + 1
            x1, y1, x2, y2 = x1x2y1y2(x,y,Length)           
            E = 4-2*((Config[x,y]==Config[x1,y]) + (Config[x,y]==Config[x2,y]) + (Config[x,y]==Config[x,y1])+ (Config[x,y]==Config[x,y2])) 
            if E >= 0
                Config[x,y] = !Config[x,y] # Flip
            else
                if E == -4
                    probability = list_exp[1]
                else
                    probability = list_exp[2]
                end      
                if rand(Float64) < probability
                    Config[x,y] = !Config[x,y] # Flip
                end
            end
        end
    end
end

In [ ]:
function dynamics_tw(Config::Array{Bool}, Temperature, Time::Int64)
    Length = size(Config,1)
    list_exp = exp.(2*[-4 -2] / Temperature)

    for time = 1:Time
        for x = 1:Length
            for y = 1:Length
                x1, y1, x2, y2 = x1x2y1y2(x,y,Length)
                E = 4-2*((Config[x,y]==Config[x1,y]) + (Config[x,y]==Config[x2,y]) + (Config[x,y]==Config[x,y1])+ (Config[x,y]==Config[x,y2])) 
                if E >= 0
                    Config[x,y] = !Config[x,y] # Flip
                else
                    if E == -4
                        probability = list_exp[1]
                    else
                        probability = list_exp[2]
                    end      
                    if rand(Float64) < probability
                        Config[x,y] = !Config[x,y] # Flip
                    end
                end
            end
        end
    end
end

In [ ]:
function stream_rs(Config::Array{Bool}, Temperature, num_sample::Int64, num_sample_burn::Int64, period_sample::Int64)::Tuple{Vector{Float64},Vector{Float64}}
    Length = size(Config,1)
    list_exp = exp.(2*[-4 -2] / Temperature)

    vec_magnetization = zeros(num_sample)
    vec_energy = zeros(num_sample)

    for time = 1:num_sample_burn
        for iteration = 1:Length^2
            x = rand(1:Length)
            y = rand(1:Length)
            x1, y1, x2, y2 = x1x2y1y2(x,y,Length)
            E = 4-2*((Config[x,y]==Config[x1,y]) + (Config[x,y]==Config[x2,y]) + (Config[x,y]==Config[x,y1])+ (Config[x,y]==Config[x,y2])) 
            
            if E >= 0
                Config[x,y] = !Config[x,y] # Flip
            else
                if E == -4
                    probability = list_exp[1]
                else
                    probability = list_exp[2]
                end      
                if rand(Float64) < probability
                    Config[x,y] = !Config[x,y] # Flip
                end
            end
        end
    end
    magnetization = 0
    magnetization2 = 0
    for time = 1:num_sample
        for iteration = 1:Length^2*period_sample
            x = rand(1:Length)
            y = rand(1:Length)
            x1, y1, x2, y2 = x1x2y1y2(x,y,Length)      
            E = 4-2*((Config[x,y]==Config[x1,y]) + (Config[x,y]==Config[x2,y]) + (Config[x,y]==Config[x,y1])+ (Config[x,y]==Config[x,y2])) 
            if E >= 0
                Config[x,y] = !Config[x,y] # Flip
            else
                if E == -4
                    probability = list_exp[1]
                else
                    probability = list_exp[2]
                end      
                if rand(Float64) < probability
                    Config[x,y] = !Config[x,y] # Flip
                end
            end
        end

        vec_magnetization[time] = -Length^2 + 2*sum(Config)

        energy_sample = 0
        for x=1:Length
            for y=1:Length
                x1, y1, x2, y2 = x1x2y1y2(x,y,Length)
                energy_sample += 4-2*((Config[x,y]==Config[x1,y]) + (Config[x,y]==Config[x2,y]) + (Config[x,y]==Config[x,y1])+ (Config[x,y]==Config[x,y2])) 
            end
        end
        vec_energy[time] = energy_sample
    end
    
    return vec_magnetization, vec_energy
end

In [ ]:
function stream_tw(Config::Array{Bool}, Temperature, num_sample::Int64, num_sample_burn::Int64, period_sample::Int64)::Tuple{Vector{Float64},Vector{Float64}}
    Length = size(Config,1)
    list_exp = exp.(2*[-4 -2] / Temperature)

    vec_magnetization = zeros(num_sample)
    vec_energy = zeros(num_sample)

    energy = 0
    for x=1:Length
        for y=1:Length
            x1, y1, x2, y2 = x1x2y1y2(x,y,Length)       
            energy += 4-2*((Config[x,y]==Config[x1,y]) + (Config[x,y]==Config[x2,y]) + (Config[x,y]==Config[x,y1])+ (Config[x,y]==Config[x,y2])) 
        end
    end
    
    for time = 1:num_sample_burn
        for x = 1:Length
            for y = 1:Length
                x1, y1, x2, y2 = x1x2y1y2(x,y,Length)
                E = 4-2*((Config[x,y]==Config[x1,y]) + (Config[x,y]==Config[x2,y]) + (Config[x,y]==Config[x,y1])+ (Config[x,y]==Config[x,y2])) 
                if E >= 0
                    Config[x,y] = !Config[x,y] # Flip
                    energy -= 2*E
                else
                    if E == -4
                        probability = list_exp[1]
                    else
                        probability = list_exp[2]
                    end      
                    if rand(Float64) < probability
                        Config[x,y] = !Config[x,y] # Flip
                        energy -= 2*E
                    end
                end
            end
        end
    end
    for time = 1:num_sample
        for iteration = 1:period_sample
            for x = 1:Length
                for y = 1:Length
                    x1, y1, x2, y2 = x1x2y1y2(x,y,Length)
                    E = 4-2*((Config[x,y]==Config[x1,y]) + (Config[x,y]==Config[x2,y]) + (Config[x,y]==Config[x,y1])+ (Config[x,y]==Config[x,y2])) 
                    if E >= 0
                        Config[x,y] = !Config[x,y] # Flip
                        energy -= 2*E
                    else
                        if E == -4
                            probability = list_exp[1]
                        else
                            probability = list_exp[2]
                        end      
                        if rand(Float64) < probability
                            Config[x,y] = !Config[x,y] # Flip
                            energy -= 2*E
                        end
                    end
                end
            end
        end
        vec_magnetization[time] = -Length^2 + 2*sum(Config)
        vec_energy[time] = energy
    end
    
    return vec_magnetization, vec_energy
end

In [ ]:
function quantity_tw(Config::Array{Bool}, Temperature, num_sample::Int64, num_sample_burn::Int64, period_sample::Int64)::Tuple{Float64,Float64,Float64,Float64}
    Length = size(Config,1)
    list_exp = exp.(2*[-4 -2] / Temperature)

    for time = 1:num_sample_burn
        for x = 1:Length
            for y = 1:Length
                x1, y1, x2, y2 = x1x2y1y2(x,y,Length)
                E = 4-2*((Config[x,y]==Config[x1,y]) + (Config[x,y]==Config[x2,y]) + (Config[x,y]==Config[x,y1])+ (Config[x,y]==Config[x,y2])) 
                if E >= 0
                    Config[x,y] = !Config[x,y] # Flip
                else
                    if E == -4
                        probability = list_exp[1]
                    else
                        probability = list_exp[2]
                    end      
                    if rand(Float64) < probability
                        Config[x,y] = !Config[x,y] # Flip
                    end
                end
            end
        end
    end
    magnetization = 0.0
    magnetization2 = 0.0
    energy = 0.0
    energy2 = 0.0
    for time = 1:num_sample
        for iteration = 1:period_sample
            for x = 1:Length
                for y = 1:Length
                    x1, y1, x2, y2 = x1x2y1y2(x,y,Length)
                    E = 4-2*((Config[x,y]==Config[x1,y]) + (Config[x,y]==Config[x2,y]) + (Config[x,y]==Config[x,y1])+ (Config[x,y]==Config[x,y2])) 
                    if E >= 0
                        Config[x,y] = !Config[x,y] # Flip
                    else
                        if E == -4
                            probability = list_exp[1]
                        else
                            probability = list_exp[2]
                        end      
                        if rand(Float64) < probability
                            Config[x,y] = !Config[x,y] # Flip
                        end
                    end
                end
            end
        end
        magnetization += abs(-Length^2 + 2*sum(Config))/num_sample
        magnetization2 += (-Length^2 + 2*sum(Config))^2/num_sample

        # using for-loop is faster than using circshift
        energy_sample = 0
        for x=1:Length
            for y=1:Length
                x1, y1, x2, y2 = x1x2y1y2(x,y,Length)
                energy_sample += 4-2*((Config[x,y]==Config[x1,y]) + (Config[x,y]==Config[x2,y]) + (Config[x,y]==Config[x,y1])+ (Config[x,y]==Config[x,y2])) 
            end
        end
        energy += energy_sample/num_sample
        energy2 += energy_sample^2/num_sample
    end
    
    return magnetization, (magnetization2-magnetization^2), energy, (energy2-energy^2)
end

In [ ]:
function stream_glauber_rs(Config::Array{Bool}, Temperature, num_sample::Int64, num_sample_burn::Int64, period_sample::Int64)::Tuple{Vector{Float64},Vector{Float64}}
    Length = size(Config,1)

    vec_magnetization = zeros(num_sample)
    vec_energy = zeros(num_sample)

    for time = 1:num_sample_burn
        for iteration = 1:Length^2
            x = rand(1:Length)
            y = rand(1:Length)
            x1, y1, x2, y2 = x1x2y1y2(x,y,Length)
            field =  -4 + 2*(Config[x1,y] + Config[x2,y] + Config[x,y1] + Config[x,y2])
            
            if rand(Float64) < 1 /(1 + exp(2*field/Temperature))
                Config[x,y] = false
            else
                Config[x,y] = true
            end
        end
    end

    for time = 1:num_sample
        for iteration = 1:Length^2*period_sample
            x = rand(1:Length)
            y = rand(1:Length)
            x1, y1, x2, y2 = x1x2y1y2(x,y,Length)
            field =  -4 + 2*(Config[x1,y] + Config[x2,y] + Config[x,y1] + Config[x,y2])
            
            if rand(Float64) < 1 /(1 + exp(2*field/Temperature))
                Config[x,y] = false
            else
                Config[x,y] = true
            end
        end

        vec_magnetization[time] = -Length^2 + 2*sum(Config)

        energy_sample = 0
        for x=1:Length
            for y=1:Length
                x1, y1, x2, y2 = x1x2y1y2(x,y,Length)
                energy_sample += 4-2*((Config[x,y]==Config[x1,y]) + (Config[x,y]==Config[x2,y]) + (Config[x,y]==Config[x,y1])+ (Config[x,y]==Config[x,y2])) 
            end
        end
        vec_energy[time] = energy_sample
    end
    
    return vec_magnetization, vec_energy
end

In [ ]:
function stream_glauber_tw(Config::Array{Bool}, Temperature, num_sample::Int64, num_sample_burn::Int64, period_sample::Int64)::Tuple{Vector{Float64},Vector{Float64}}
    Length = size(Config,1)

    vec_magnetization = zeros(num_sample)
    vec_energy = zeros(num_sample)

    for time = 1:num_sample_burn
        for x = 1:Length
            for y = 1:Length
                x1, y1, x2, y2 = x1x2y1y2(x,y,Length)
                field =  -4 + 2*(Config[x1,y] + Config[x2,y] + Config[x,y1] + Config[x,y2])
                if rand(Float64) < 1 /(1 + exp(2*field/Temperature))
                    Config[x,y] = 0
                else
                    Config[x,y] = 1
                end
            end
        end
    end
    
    for time = 1:num_sample
        for iteration = 1:period_sample
            for x = 1:Length
                for y = 1:Length              
                    x1, y1, x2, y2 = x1x2y1y2(x,y,Length)
                    field =  -4 + 2*(Config[x1,y] + Config[x2,y] + Config[x,y1] + Config[x,y2])
                    if rand(Float64) < 1 /(1 + exp(2*field/Temperature))
                        Config[x,y] = 0
                    else
                        Config[x,y] = 1
                    end
                end
            end
        end

        vec_magnetization[time] = -Length^2 + 2*sum(Config)

        energy_sample = 0
        for x=1:Length
            for y=1:Length
                x1, y1, x2, y2 = x1x2y1y2(x,y,Length)
                energy_sample += 4-2*((Config[x,y]==Config[x1,y]) + (Config[x,y]==Config[x2,y]) + (Config[x,y]==Config[x,y1])+ (Config[x,y]==Config[x,y2])) 
            end
        end
        vec_energy[time] = energy_sample
    end
    
    return vec_magnetization, vec_energy
end

Functions related with autocorrelation.

**autocorrelation** (data, k): 	*k-step autocorrelation of data*

**autocorrelation_function** (data, kmax):	*Returns 1- to kmax-step autocorrelation of data*

**autocorrelation_time** (data, kmax):	*Returns autocorrelation time, which is the first time that autocorrelation is less than exp(-1)*

In [ ]:
function autocorrelation(data, k::Int64)::Float64
    n = length(data)
    mu = mean(data)
    variance = var(data)
    
    corr = 0.0
    for t = 1:(n - k)
        corr += (data[t] - mu) * (data[t + k] - mu)
    end
    
    return corr / (variance * (n - k))
end
        
function autocorrelation_function(data, kmax::Int64)::Vector{Float64}
    corr = Array(Float64, kmax)
    for k = 0:kmax-1
        corr[k+1] = autocorrelation(data, k)
    end
    return corr
end
    
function autocorrelation_time(data, kmax::Int64)::Float64
    corr = Array(Float64, kmax)
    for k = 0:kmax-1
        corr[k+1] = autocorrelation(data, k)
    end
    if length(find(corr.<exp(-1))) == 0
        return Inf
    end
    t2 = find(corr.<exp(-1))[1]
    t1 = t2 -1
    return (t2*(corr[t1]-exp(-1)) + t1*(-corr[t2]+exp(-1))) /(corr[t1]-corr[t2])
end